In [8]:
from jira.client import JIRA
options = {'server': "https://hibernate.atlassian.net/"}
jira = JIRA(options, basic_auth=("ossama.kordoghli@ensi-uma.tn", "cOchFOzwM9kPculNOs1p343D"))
# d,l=productivity_jira("ANN",jira)

In [4]:
issues = jira.search_issues('project={}'.format("ANN"),  0,100)

In [5]:
ticket = jira.search_issues('project=ANN', 1,1)
ticket

[<JIRA Issue: key='ANN-805', id='19536'>]

In [6]:
len(ticket[0].fields.customfield_10050)

2

In [7]:
print(issues[1].fields.created)
print(issues[1].fields.resolutiondate)

2009-02-23T10:06:30.448-0800
2009-05-05T14:14:09.040-0700


In [8]:
issues[0].fields.resolutiondate.split('.')

['2009-03-31T06:49:25', '767-0700']

In [9]:
from datetime import datetime,date,timedelta
d1=datetime.strptime((issues[59].fields.resolutiondate.split('.'))[0],'%Y-%m-%dT%H:%M:%S')
d1.year
print(datetime.today()-timedelta(7))

2022-04-13 21:36:02.064805


In [10]:
issues[0].key

'ANN-813'

In [11]:
def productivity_jira(project,jira):
    data=[]
    #data=[{}]
    last_ticket=""
    persons=[]
    i=0
    test=False
    size = 100
    initial = 0
    while True:
        start= initial*size
        issues = jira.search_issues('project={}'.format(project), start,size)
        if len(issues) == 0:
            break
        initial += 1

        if(not test):
            last_ticket=issues[0].key.split("-")[1]
            test=True
        
        for issue in issues:
            for j in range(len(issue.fields.customfield_10050)):
                if(issue.fields.customfield_10050[j].displayName not in persons):  
                    data.append({"name":"","closed tickets":0,"all tickets":0,"remain_tickets":0,"in progress":0,"tickets with issues":0})
                    persons.append(issue.fields.customfield_10050[j].displayName)
                    data[i]["name"]=issue.fields.customfield_10050[j].displayName
   
                    i+=1

                
        for issue in issues:
            for e in data :
                for j in range(len(issue.fields.customfield_10050)):
                    if(issue.fields.customfield_10050[j].displayName==e["name"]):
                        e["all tickets"]+=1
                        if(issue.fields.status.name=='Closed' or issue.fields.status.name=='Resolved') :
                            e["closed tickets"]+=1
                        elif(issue.fields.status.name=='Open') :
                            e["in progress"]+=1
                        #elif(eli andhom mochkla fel resolution)
                            #e["tickets with issues"]+=1
                    e["remain tickets"]=e["all tickets"]-e["closed tickets"] #neksa tickets with issues
        
    return data,last_ticket

In [12]:
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np 
import warnings
warnings.filterwarnings('ignore')
import math
import pickle


In [13]:
data= pd.read_csv("./data.csv")

In [14]:
data

,Type de ticket,Priorité,Nombre de Champs personnalisés (Participants),Annee,Jour,Mois,Nombre de Composants,Nombre de Versions corrigées,Nb_heures,Type de Version
0,Tâche,1,1,2022,0,3,1,0,2,Majeur
1,Tâche,1,1,2022,6,3,1,0,2,Majeur
2,Tâche,1,1,2022,0,2,1,0,20,Majeur
3,Tâche,1,1,2022,3,1,1,0,4,Majeur
4,Tâche,1,1,2022,2,1,1,0,6,Majeur
...,...,...,...,...,...,...,...,...,...,...
102349,Bug,5,5,2019,6,3,4,4,7,Mineur
102350,Tâche,5,5,2009,6,3,3,3,7,Mineur
102351,Sous-tâche,4,5,2019,7,7,3,4,2,Majeur
102352,Tâche,4,5,2016,6,3,3,3,6,Patch


In [15]:
filehandler2 = open('../IA/time', 'rb')

loaded_model = pickle.load(filehandler2)
filehandler2.close()

In [16]:
file = open('../IA/encoderTypeTicket', 'rb')

Type_ticket = pickle.load(file)
file.close()

In [17]:
from sklearn.preprocessing import LabelEncoder
Trans = LabelEncoder()
data["Type de ticket"]=Type_ticket.transform(data["Type de ticket"])


In [18]:
file1 = open('../IA/encoderTypeVersion', 'rb')

Type_Version = pickle.load(file1)
file1.close()

In [19]:
from sklearn.preprocessing import LabelEncoder
Trans = LabelEncoder()
data["Type de Version"]=Type_Version.transform(data["Type de Version"])

In [20]:
# preds=loaded_model.predict(data[["Type de ticket","Priorité","Nombre de Champs personnalisés (Participants)","Annee","Jour","Mois","Nombre de Composants","Nombre de Versions corrigées","Type de Version"]])
preds=loaded_model.predict([[1,1,1,2022,1,3,1,10,0]])

In [21]:
preds[0]

13.62

In [22]:
data

,Type de ticket,Priorité,Nombre de Champs personnalisés (Participants),Annee,Jour,Mois,Nombre de Composants,Nombre de Versions corrigées,Nb_heures,Type de Version
0,10,1,1,2022,0,3,1,0,2,0
1,10,1,1,2022,6,3,1,0,2,0
2,10,1,1,2022,0,2,1,0,20,0
3,10,1,1,2022,3,1,1,0,4,0
4,10,1,1,2022,2,1,1,0,6,0
...,...,...,...,...,...,...,...,...,...,...
102349,1,5,5,2019,6,3,4,4,7,1
102350,10,5,5,2009,6,3,3,3,7,1
102351,7,4,5,2019,7,7,3,4,2,0
102352,10,4,5,2016,6,3,3,3,6,2


In [119]:
def prediction_time(project,jira):
    data=[]
    time=[]
    #data=[{}]
    last_ticket=""
    persons=[]
    arret=False
    i=0
    test=False
    size = 100
    initial = 0
    while True:
        start= initial*size
        issues = jira.search_issues('project={}'.format(project), start,size)
        if len(issues) == 0:
            break
        initial += 1

        if(not test):
            last_ticket=issues[0].key.split("-")[1]
            test=True
        
        for issue in issues:
            if (issue.fields.resolutiondate ): 
                d1=datetime.strptime((issue.fields.resolutiondate.split('.'))[0],'%Y-%m-%dT%H:%M:%S')
                if (d1>datetime.today()-timedelta(20)):
                    dict={"Type de ticket":0,"Priority":0,"Nombre de participant":0,"Annee":0,"Jour":0,"Mois":0,"Nombre de Composants":0,"Nombre de Versions corrigées":0,"Type de Version":"Patch"}
                    dict["Type de ticket"]=issue.fields.issuetype.name 
                    dict["Priority"]=issue.fields.priority.name 
                    dict["Nombre de participant"]=len(issue.fields.customfield_10050)
                    dict["Annee"]=d1.year 
                    dict["Mois"]=d1.month
                    dict["Jour"]=d1.day 
                    dict["Nombre de Composants"]=len(issue.fields.components)
                    dict["Nombre de Versions corrigées"]=len(issue.fields.components)
                    if (len(issue.fields.fixVersions) != 0): 
                        print(issue.fields.fixVersions)
                        if (len((issue.fields.fixVersions)[0].name.split(".")) ==3):
                            if (((issue.fields.fixVersions)[0].name.split("."))[0] != 0 ):
                                dict["Type de Version"]='Majeur' 
                            elif (((issue.fields.fixVersions)[0].name.split("."))[0] == 0 and ((issues[0].fields.fixVersions)[1].name.split("."))[0] != 0):
                                dict["Type de Version"]='Mineur'
                            else :
                                dict["Type de Version"]='Patch' 
                    data.append(dict)
                    d2=datetime.strptime((issue.fields.created.split('.'))[0],'%Y-%m-%dT%H:%M:%S')
                    time.append(((d1-d2).seconds/3600 + (d1-d2).days*24))
                else: 
                    arret=True
                    break   
        if (arret == True):
            break
    return data,time,last_ticket

In [120]:
def preparation_data(project,jira):
    hh=prediction_time(project,jira)
    for i in hh[0]:
        if(i['Type de ticket']=="Bug"):
            i['Type de ticket']="Bug"
        elif(i['Type de ticket']=="Task"):
            i['Type de ticket']="Tâche"

        elif(i['Type de ticket']=="Improvement"):
            i['Type de ticket']="Amélioration"
            
        elif(i['Type de ticket']=="Sub-task"):
            i['Type de ticket']="Sous-tâche"

        elif(i['Type de ticket']=="New Feature"):
            i['Type de ticket']="Nouvelle fonctionnalité"
        
        elif(i['Type de ticket']=="Deprecation"):
            i['Type de ticket']="Deprecation"
        elif(i['Type de ticket']=="Epic"):
            i['Type de ticket']="Epic"
        elif(i['Type de ticket']=="Remove Feature"):
            i['Type de ticket']="Remove Feature"
        elif(i['Type de ticket']=="Patch"):
            i['Type de ticket']="Patch"  

        elif(i['Type de ticket']=="Technical task"):
            i['Type de ticket']="Technical task"
        elif(i['Type de ticket']=="Story"):
            i['Type de ticket']="Story" 


    for i in hh[0]:
        if (i['Priority']=="Trivial"):
            i['Priority']=1
        elif (i['Priority']=="Minor"):
            i['Priority']=2
        elif (i['Priority']=="Major"):
            i['Priority']=3
        elif (i['Priority']=="Critical"):
            i['Priority']=4
        elif (i['Priority']=="Blocker"):
            i['Priority']=5
    
    file = open('../IA/encoderTypeTicket', 'rb')
    Type_ticket = pickle.load(file)
    file.close()
    for i in hh[0]:
        i['Type de ticket']=(Type_ticket.transform([i['Type de ticket']]))[0]   
    
    file1 = open('../IA/encoderTypeVersion', 'rb')

    Type_Version = pickle.load(file1)
    file1.close()
    for i in hh[0]:
        i['Type de Version']=(Type_Version.transform([i['Type de Version']]))[0]
    
    filehandler2 = open('../IA/time', 'rb')

    loaded_model = pickle.load(filehandler2)
    filehandler2.close()
    for i in hh[0] :
        preds=loaded_model.predict([list(i.values())])
        i["prediction"]=preds[0]
    return hh

In [121]:
from datetime import datetime
import calendar 

def findDay(date): 
    born = datetime.strptime(date, '%d %m %Y').weekday() 
    return (calendar.day_name[born]) 
def final_output(project,jira):
    hh=preparation_data(project,jira)
    L=[]
    for i in range(len(hh[0])-1):
        dict={"Jour":"Lundi", "Prediction":0 ,"real":0}
        t =findDay(str(hh[0][i]["Jour"])+' '+str(hh[0][i]["Mois"])+' '+str(hh[0][i]["Annee"]))
        dict["Jour"]= t
        dict["Prediction"] =hh[0][i]['prediction']
        dict['real']=hh[1][i]
        L.append(dict)
    
    final={"Monday":[0,0,0], "Tuesday":[0,0,0], "Wednesday":[0,0,0], "Thursday":[0,0,0],"Friday":[0,0,0], "Saturday":[0,0,0], "Sunday":[0,0,0]}
    for i in L:
        if i['Jour']=="Monday":
            final["Monday"][0] = final["Monday"][0]+i['Prediction']
            final["Monday"][1] = final["Monday"][1]+i["real"]
            final["Monday"][2] +=1 
        if i['Jour']=="Tuesday":
            final["Tuesday"][0] += i['Prediction']
            final["Tuesday"][1] += i["real"]
            final["Tuesday"][2] +=1 
        if i['Jour']=="Wednesday":
            final["Wednesday"][0] += i['Prediction']
            final["Wednesday"][1] += i["real"]
            final["Wednesday"][2] +=1 
        if i['Jour']=="Thursday":
            final["Thursday"][0] += i['Prediction']
            final["Thursday"][1] += i["real"]
            final["Thursday"][2] +=1 
        if i['Jour']=="Friday":
            final["Friday"][0] += i['Prediction']
            final["Friday"][1] += i["real"]
            final["Friday"][2] +=1 
        if i['Jour']=="Saturday":
            final["Saturday"][0] += i['Prediction']
            final["Saturday"][1] += i["real"]
            final["Saturday"][2] +=1 
        if i['Jour']=="Sunday":
            final["Sunday"][0] += i['Prediction']
            final["Sunday"][1] += i["real"]
            final["Sunday"][2] +=1 

    for i in final.keys():
        if (final[i][2] != 0):
            final[i][0]=final[i][0]/final[i][2]
            final[i][1]=final[i][1]/final[i][2]

    return final,hh[2]
    


In [122]:
final_output("HHH",jira)

[<JIRA Version: name='5.6.8', id='32056'>, <JIRA Version: name='6.0.Next', id='32061'>]
[<JIRA Version: name='6.0.Next', id='32061'>]
[<JIRA Version: name='6.0.Next', id='32061'>]
[<JIRA Version: name='6.0.Next', id='32061'>]
[<JIRA Version: name='6.0.Next', id='32061'>]
[<JIRA Version: name='6.0.Next', id='32061'>]
[<JIRA Version: name='6.0.Next', id='32061'>]
[<JIRA Version: name='6.0.Next', id='32061'>]
[<JIRA Version: name='6.0.Next', id='32061'>]
[<JIRA Version: name='6.0.Next', id='32061'>]
[<JIRA Version: name='6.0.Next', id='32061'>]
[<JIRA Version: name='6.0.Next', id='32061'>]
[<JIRA Version: name='6.0.Next', id='32061'>]
[<JIRA Version: name='6.0.Next', id='32061'>]
[<JIRA Version: name='6.0.Next', id='32061'>]
[<JIRA Version: name='6.0.Next', id='32061'>]
[<JIRA Version: name='6.0.Next', id='32061'>]
[<JIRA Version: name='6.0.Next', id='32061'>]


({'Monday': [74.15267346938775, 39.986785714285716, 7],
  'Tuesday': [47.19699999999999, 22.983666666666668, 5],
  'Wednesday': [29.11166666666667, 2.8175, 4],
  'Thursday': [70.23583333333333, 29.189305555555556, 2],
  'Friday': [38.472500000000004, 22.130185185185184, 3],
  'Saturday': [0, 0, 0],
  'Sunday': [0, 0, 0]},
 '15218')

In [6]:
import datetime
import calendar
dt = datetime.datetime(year=2014, month=5, day=31)
d = calendar.monthrange(dt.year,dt.month+1)[1] 
print (dt+datetime.timedelta(days=d+1))

2014-07-01 00:00:00


In [7]:
from datetime import date

today = date.today()
print("Today's date:", today)

Today's date: 2022-04-21


In [16]:
from datetime import datetime,date,timedelta
import pickle
def prediction_nombre_ticket(project,jira):
    issues = jira.search_issues('project={}'.format(project), 0,1)
    last_ticket=issues[0].key.split("-")[1]
    liste=["Jan","Feb","March","Apr","Mai","Jun","Jul","Aug","Sep","Oct","Nov","Dec"]
    dict={}
    today = date.today()
    k = today.month 
    while (k != 13):
        dict[liste[k-1]]=0
        k+=1 
    dict
    filehandler2 = open('../IA/ticketParMois', 'rb')

    loaded_model = pickle.load(filehandler2)
    filehandler2.close()
    # preds=loaded_model.predict([[0,today.year,today.month,today.day]])
    for j in range(len(dict.keys())):
        s =0
        for i in range(31):
            if((datetime(year=today.year, month=today.month+j, day=1)+timedelta(i)).month == today.month+j):
                s=s+abs((loaded_model.predict([[0,today.year,today.month,i]]))[0])
                s=s+(loaded_model.predict([[1,today.year,today.month,i]]))[0]
                s=s+(loaded_model.predict([[2,today.year,today.month,i]]))[0]
                s=s+(loaded_model.predict([[3,today.year,today.month,i]]))[0]
                s=s+(loaded_model.predict([[4,today.year,today.month,i]]))[0]
                s=s+(loaded_model.predict([[5,today.year,today.month,i]]))[0]
                s=s+(loaded_model.predict([[6,today.year,today.month,i]]))[0]
                s=s+(loaded_model.predict([[7,today.year,today.month,i]]))[0]
                s=s+(loaded_model.predict([[8,today.year,today.month,i]]))[0]
                s=s+(loaded_model.predict([[9,today.year,today.month,i]]))[0]
                s=s+(loaded_model.predict([[10,today.year,today.month,i]]))[0]
        dict[list(dict.keys())[j]]=int(s)
    return dict,last_ticket

    
    



In [18]:
prediction_nombre_ticket("HHH",jira)

C:\Users\oussa\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.0.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\oussa\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator RandomForestRegressor from version 1.0.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\oussa\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but 

({'Apr': 2144,
  'Mai': 2215,
  'Jun': 2144,
  'Jul': 2215,
  'Aug': 2215,
  'Sep': 2144,
  'Oct': 2215,
  'Nov': 2144,
  'Dec': 2215},
 '15222')

In [15]:
last_ticket

'15222'

In [28]:
int(preds[0])

5

In [45]:
list(dict.keys())[0]

'Apr'

In [15]:
import warnings
warnings.filterwarnings('ignore')
import math
import pickle
def Prediction_Number_Participants(issue_type,priority,annee,jour,mois,number_components,number_versions,version_type):
    model_cb = CatBoostClassifier(task_type='GPU', iterations=100, random_state = 2021, eval_metric="Accuracy",boost_from_average=False)
    model_participant=model_cb.load_model("../IA/participant")
    
    file = open('../IA/encoderTypeTicket', 'rb')
    Type_ticket = pickle.load(file)

    issue_type_enc=(Type_ticket.transform([issue_type]))[0]

    file = open('../IA/encoderTypeVersion', 'rb')
    Type_version = pickle.load(file)

    version_type_enc=(Type_version.transform([version_type]))[0]

    return (model_participant.predict([[issue_type_enc,int(priority),annee,jour,mois,int(number_components),int(number_versions),version_type_enc]]))[0][0]


In [16]:
Prediction_Number_Participants("Bug",1,2022,22,5,1,1,"Majeur")

1

In [7]:
filehandler2 = open('../IA/ticketParMois', 'rb')
loaded_model = pickle.load(filehandler2)
filehandler2.close()

In [1]:
from catboost import CatBoostClassifier



In [12]:
file = open('../IA/encoderTypeTicket', 'rb')
Type_ticket = pickle.load(file)
issue_type_enc=(Type_ticket.transform(['Bug']))[0]

In [13]:
issue_type_enc

1